In [12]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(1, "drive/MyDrive/workflow")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install pymatgen transformers datasets nltk ipywidgets seqeval[gpu]
!jupyter nbextension enable --py widgetsnbextension

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [14]:
import numpy as np
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
import torch
from torch.utils.data import DataLoader, random_split
from transformers import BertTokenizerFast
import psie

from seqeval.metrics import classification_report
import os
from torch import cuda
import nltk
nltk.download("punkt", quiet=True)

import re
from pymatgen.core import Composition

import json

In [15]:
device = "cuda" if cuda.is_available() else "cpu"
print(device)

cuda


In [16]:
radio_buttons = widgets.RadioButtons(
    options=["Band Gap", "Curie Temperature"], value="Band Gap", description=''
)
print("Extraction Target: ")
display(radio_buttons)

Extraction Target: 


RadioButtons(options=('Band Gap', 'Curie Temperature'), value='Band Gap')

In [23]:
if radio_buttons.value == "Curie Temperature":
  extr_target = "Tc"
elif radio_buttons.value == "Band Gap":
  extr_target = "Gap"


BERT_VERSION = 'm3rg-iitd/matscibert'
MAX_LEN = 256
MAIN_DIR = os.path.join("drive", "MyDrive", "workflow")
MODEL_DIR = os.path.join("models", extr_target, "classifier")
CORPUS = os.path.join("corpus", extr_target, "corpus_sentences.json")
OUTPUT="relevant_sentences"

In [24]:
from datasets import load_dataset
dataset = load_dataset('json', data_files=os.path.join(MAIN_DIR, CORPUS))["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-18844bab45978139/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
model = psie.classifier.BertClassifier()

model.load_state_dict(torch.load(os.path.join(MAIN_DIR, MODEL_DIR+ ".pt")))
model.to(device)

Some weights of the model checkpoint at m3rg-iitd/matscibert were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at m3rg-iitd/matscibert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
Y

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [28]:
tokenizer = BertTokenizerFast.from_pretrained(BERT_VERSION)

def encode(paper):
  return tokenizer(paper["sentence"], truncation=True, max_length=MAX_LEN, padding="max_length")

dataset = dataset.map(encode, batched=True)
dataset.set_format(type="torch", columns=["source", "sentence", "input_ids", "attention_mask"])
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)

Map:   0%|          | 0/697 [00:00<?, ? examples/s]

In [29]:
pred = model.predict(dataset_loader, device)

In [30]:
predictions = []

for i in range(len(pred)):
  predictions.append(np.argmax(pred[i].cpu().numpy()))

In [31]:
print(np.sum(predictions), "/", len(dataset))

290 / 697


In [32]:
filtered_sentences = {"sentence": [], "source":[]}

for i in range(len(predictions)):
  if predictions[i] == 1:
    filtered_sentences["sentence"].append((dataset[i]["sentence"]))
    filtered_sentences["source"].append((dataset[i]["source"]))

with open(os.path.join(MAIN_DIR, "corpus", extr_target, OUTPUT+".json"), "w") as f:
  json.dump(filtered_sentences, f)